#### This coding assignment is to Simulate a 2D Lid-Driven Cavity Flow, for stookes flow

In [1]:
from matplotlib import pyplot
import numpy
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size']=16

In [2]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

Below we import common codes used in Module 5 to help with our analysis:

In [3]:
import numpy 
from matplotlib import pyplot, cm 
from mpl_toolkits.mplot3d import Axes3D 

 
def plot_3D(x, y, p, elev=30, azi=45):
    fig = pyplot.figure(figsize=(11,7), dpi=100) 
    ax = fig.gca(projection='3d') 
    X,Y = numpy.meshgrid(x,y) 
    surf = ax.plot_surface(X,Y,p[:], rstride=1, cstride=1, cmap=cm.viridis, 
    linewidth=0, antialiased=False) 
    ax.set_xlabel('$x$') 
    ax.set_ylabel('$y$') 
    ax.set_zlabel('$z$') 
    ax.view_init(elev,azi) 
 
 

def L1norm(new, old):
    norm = numpy.sum(numpy.abs(new-old))
    return norm

It appears that our governing equations are a laplace equation and a Poisson equation:

Laplace: $$\triangledown^{2}u=0 \longrightarrow \frac{\partial^{2}u}{\partial x^{2}}+\frac{\partial^{2}u}{\partial y^{2}}=0$$

Poisson: $$\triangledown^{2}u=f  \longrightarrow \frac{\partial^{2}u}{\partial x^{2}}+\frac{\partial^{2}u}{\partial y^{2}}=f$$

Specifically we have:

 $$\triangledown^{2}w=0 \longrightarrow \frac{\partial^{2}w}{\partial x^{2}}+\frac{\partial^{2}w}{\partial y^{2}}=0$$

 $$\triangledown^{2}\psi=w  \longrightarrow \frac{\partial^{2} \psi}{\partial x^{2}}+\frac{\partial^{2} \psi}{\partial y^{2}}=w$$

I like to first look at the boundary conditions for this problem, since, they define the entire problem. It looks like, from notebook 5, the LHS, RHS, and Bottom BC's are all $\psi =0$ and insulated with $\frac{\partial \psi}{\partial x(y)}=0$. And the top BC is $\psi=0, \frac{\partial \psi}{\partial y}=0$. So all four BC's have both Nuemann and Dirichlet boundary conditions. The below cell tries to reproduce them in code form:

we are given a relation between $w$ and $\psi$ for the top surface Boundary condition:

$$w_{i,j}=\frac{-1}{2\Delta y^{2}}(8\psi_{i,j-1} - \psi_{i,j-2}) - \frac{3u_{j}}{\Delta y}$$

where u is the speed of the top surface at point y=j. Which is constant at 1 if j=the top.

From Notebook 1, the process for coding a Laplace equation seems to be;

1. define and create meshgrid that has the analytical solution (P_an)
2. define a function for using the L2 norm to compare two succesive potnetial fields to ensure we meet our relation criteria target. (L2_error)
3. Define and write a function to applice the Jacobi method of iteration for the Laplace equation (Laplace2d)


In my case,I will use the L2Norm function given by the assignment.

Question for myself: Should I use 1st or 2nd order Nuemann BC statements?

In [4]:
#intial parameters

nx = 41
ny = 41

l=1
h=1

dx=l/(nx-1)
dy=h/(ny-1)


l1_target = 1e-6


In [5]:
#mesh
x=numpy.linspace(0,l,nx)
y=numpy.linspace(0,h,ny)

w=numpy.zeros((ny,nx))
psi=numpy.zeros((ny,nx))

In [6]:
print(w)
print(psi)
print(psi[0,1])

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
0.0


And now the answers:

In [10]:

wnorm=1
psinorm=1
u=numpy.linspace(0,1, nx)
u[:]=1                    #array for the speed at the top of the surface
 

while wnorm > l1_target:
    
    while psinorm>l1_target:
        
        wL = w.copy()
        psiP=psi.copy()
        
        #Laplace
        w[1:-1,1:-1] = .25 * (wL[1:-1,2:] + wL[1:-1, :-2] + wL[2:, 1:-1] + wL[:-2, 1:-1])
 
        #poisson
        psi[1:-1,1:-1] = (0.25)*(psiP[1:-1,2:] + psiP[1:-1,:-2] + psiP[2:,1:-1] + psiP[:-2,1:-1] + wL[1:-1,1:-1]*dx**2)
        
        #w BC's
        
        
        
        w[-1,:]= -(1/(2*dy**2))*(8*psi[-2,:] - psi[-3,1:]) - 3/(dy)     #y=h (top)
        
        w[0,1:-1]= -(1/(2*dy**2))*(5*psi[-1,1:-1]+psi[-3,1:-1]-8*psi[-2,1:-1])        #y=0 (Bottom) 
        w[1:-1,-1]= -(1/(2*dx**2))*(5*psi[1:-1,-1]+psi[1:-1,-3]-8*psi[1:-1,-2])       #x=l (right surface)
        w[1:-1,0]=  -(1/(2*dx**2))*(5*psi[1:-1,-1]+psi[1:-1,-3]-8*psi[1:-1,-2])       #x=0 (left surface)
    
        psinorm=L1norm(psiP,psi)
        wnorm = L1norm(wL, w)
        
    

C:\Users\ctiu\Anaconda3\lib\site-packages\ipykernel\__main__.py:16: RuntimeWarning: overflow encountered in add
C:\Users\ctiu\Anaconda3\lib\site-packages\ipykernel\__main__.py:27: RuntimeWarning: invalid value encountered in subtract
C:\Users\ctiu\Anaconda3\lib\site-packages\ipykernel\__main__.py:28: RuntimeWarning: invalid value encountered in subtract
C:\Users\ctiu\Anaconda3\lib\site-packages\ipykernel\__main__.py:20: RuntimeWarning: invalid value encountered in subtract


In [8]:
max_psi=numpy.max(psi)
max_w=numpy.max(w)
print('Max psi is:', max_psi)
print('Max w is:', max_w)

Max psi is: 0.0
Max w is: nan


In [9]:
numpy.round(psiP[32,::8],4)

C:\Users\ctiu\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2782: RuntimeWarning: overflow encountered in multiply
  return round(decimals, out)
C:\Users\ctiu\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2782: RuntimeWarning: invalid value encountered in rint
  return round(decimals, out)


array([             -inf,  -1.49984555e+303,  -2.02799979e+301,
        -2.02799979e+301,  -1.49984555e+303,              -inf])